In [3]:
import numpy as np
import math
from scipy.special import comb
from scipy.linalg import sqrtm

# Parameters
dx = 784  # MNIST dataset feature dimension
dy = 10   # MNIST dataset label dimension (one-hot encoded)
l = 128    # Mixture degree
n = 60000 # Number of samples in the dtaset
sigma_x = 0.1132 # Standard deviation for features (example value)
sigma_y = sigma_x  # Standard deviation for labels (same as sigma_x)
T = 10000  # Number of synthetic data points to generate

# Sensitivity calculation
Delta2 = (dx / sigma_x**2) + (dy / sigma_y**2)

# Bernoulli polynomial B(l) calculation
def B(l, Delta2):
    sum_B = 0
    for i in range(l + 1):
        sum_B += (-1)**i * comb(l, i) * np.exp(i * (i - 1) * Delta2 / (2 * l**2))
    return sum_B

# G(alpha) calculation
def G(alpha, l, n, Delta2):
    sum_G = 0
    for j in range(3, alpha + 1):
        sum_G += (l / n)**j * comb(alpha, j) * math.sqrt(B(2 * (j // 2), Delta2) * B(2 * (j - j // 2), Delta2))
    return sum_G

# εα' calculation
def epsilon_alpha_prime(alpha, l, n, Delta2):
    min_term = min(4 * (np.exp(Delta2 / l**2) - 1), 2 * np.exp(Delta2 / l**2))
    term1 = (l / n)**2 * comb(alpha, 2) * min_term
    term2 = 4 * G(alpha, l, n, Delta2)
    return (1 / (alpha - 1)) * np.log(1 + term1 + term2)

# Final ε calculation
def epsilon(T, l, n, Delta2, delta=1/n):
    epsilons = []
    for alpha in range(2, 101): # Varying alpha from 2 to 100
        try:
            epsilon_alpha = epsilon_alpha_prime(alpha, l, n, Delta2)
            epsilon_total = T * epsilon_alpha + np.log(1 / delta) / (alpha - 1)
            epsilons.append(epsilon_total)
        except (OverflowError, ValueError):
            break  # Stop if an error occurs to avoid instability
    return min(epsilons)


# Calculate ε
epsilon_value = epsilon(T, l, n, Delta2)
print(f"Calculated ε: {epsilon_value}")


C:\Users\User\AppData\Local\Temp\ipykernel_10272\2386208106.py:22: RuntimeWarning: overflow encountered in exp
  sum_B += (-1)**i * comb(l, i) * np.exp(i * (i - 1) * Delta2 / (2 * l**2))
C:\Users\User\AppData\Local\Temp\ipykernel_10272\2386208106.py:22: RuntimeWarning: invalid value encountered in scalar add
  sum_B += (-1)**i * comb(l, i) * np.exp(i * (i - 1) * Delta2 / (2 * l**2))
C:\Users\User\AppData\Local\Temp\ipykernel_10272\2386208106.py:22: RuntimeWarning: overflow encountered in scalar multiply
  sum_B += (-1)**i * comb(l, i) * np.exp(i * (i - 1) * Delta2 / (2 * l**2))


Calculated ε: 14.99703895832793
